In [ ]:
import argparse
import torch
import os

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path

from PIL import Image
import requests
from io import BytesIO
from transformers import TextStreamer
from datasets import load_dataset

def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

def main(args):
    # Disable Torch initialization for faster loading
    disable_torch_init()

    # Load pretrained model
    model_name = get_model_name_from_path(args.model_path)
    tokenizer, model, image_processor, context_len = load_pretrained_model(
        args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit, device=args.device
    )

    # Determine conversation mode based on model name
    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print('[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}'.format(conv_mode, args.conv_mode, args.conv_mode))
    else:
        args.conv_mode = conv_mode

    # Copy conversation template
    conv = conv_templates[args.conv_mode].copy()
    roles = ('user', 'assistant') if "mpt" in model_name.lower() else conv.roles

    # Load the dataset
    dataset = load_dataset("abhinavl/figure2code_data", split="test")

    # Define the fixed question for all images
    fixed_question = (
        "Convert the figure you are given into a full code. Here is an example of the expected output:\n"
        "import matplotlib.pyplot as plt\n"
        "# Categories and their corresponding values\n"
        "categories = ['essay', 'soil']\n"
        "values = [1, 2]\n"
        "# Creating the bar chart\n"
        "plt.figure(figsize=(8, 5))  # Set the figure size (optional)\n"
        "plt.bar(categories, values, color='skyblue')  # Plot the bars with skyblue color\n"
        "# Adding title and labels\n"
        "plt.title('Title')  # Add a title to the chart\n"
        "plt.xlabel('Categories')  # Label for the x-axis\n"
        "plt.ylabel('Values')  # Label for the y-axis\n"
        "# Display the chart\n"
        "plt.show()"
    )

    # Process each image in the dataset
    for idx, item in enumerate(dataset):
        image = load_image(item['image'])
        image_size = image.size

        # Process image tensor
        image_tensor = process_images([image], image_processor, model.config)
        if type(image_tensor) is list:
            image_tensor = [image.to(model.device, dtype=torch.float16) for image in image_tensor]
        else:
            image_tensor = image_tensor.to(model.device, dtype=torch.float16)

        # Prepare the input with the fixed question
        if model.config.mm_use_im_start_end:
            inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + fixed_question
        else:
            inp = DEFAULT_IMAGE_TOKEN + '\n' + fixed_question

        conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        # Tokenize input and generate response
        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=image_tensor,
                image_sizes=[image_size],
                do_sample=True if args.temperature > 0 else False,
                temperature=args.temperature,
                max_new_tokens=args.max_new_tokens,
                streamer=streamer,
                use_cache=True
            )

        outputs = tokenizer.decode(output_ids[0]).strip()
        conv.messages[-1][-1] = outputs

        # Print output if in debug mode
        if args.debug:
            print("\n", {"prompt": prompt, "outputs": outputs}, "\n")
        else:
            print(f"{roles[1]}: {outputs}")

        # Save output to a .py file
        with open(f"test_{idx}.py", "w") as f:
            f.write(outputs)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-path", type=str, default="facebook/opt-350m")
    parser.add_argument("--model-base", type=str, default=None)
    parser.add_argument("--device", type=str, default="cuda")
    parser.add_argument("--conv-mode", type=str, default=None)
    parser.add_argument("--temperature", type=float, default=0.2)
    parser.add_argument("--max-new-tokens", type=int, default=512)
    parser.add_argument("--load-8bit", action="store_true")
    parser.add_argument("--load-4bit", action="store_true")
    parser.add_argument("--debug", action="store_true")
    args = parser.parse_args()
    main(args)

In [2]:

from datasets import load_dataset
# Load the dataset

dataset = load_dataset("abhinavl/figure2code_data", split="test")


/opt/anaconda3/envs/cs231n-fp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

from PIL import Image
from tqdm import tqdm


def load_image(image_file):
    if isinstance(image_file, Image.Image):
        image = image_file.convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image


for idx, item in tqdm(enumerate(dataset), total=len(dataset), desc="Processing images"):
    image = load_image(item['image'])
    image_size = image.size
    print("image_size", image_size)

Processing images:   3%|▎         | 26/1000 [00:00<00:07, 128.83it/s]

image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)


Processing images:   5%|▌         | 54/1000 [00:00<00:06, 135.31it/s]

image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)


Processing images:   8%|▊         | 82/1000 [00:00<00:06, 134.34it/s]

...

image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)


Processing images:  96%|█████████▋| 964/1000 [00:07<00:00, 136.82it/s]

image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)


Processing images: 100%|██████████| 1000/1000 [00:07<00:00, 135.42it/s]

image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
image_size (800, 500)
